In [30]:
from dotenv import load_dotenv
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, From, content
from typing import Dict
from IPython.display  import display, Markdown

In [ ]:
load_dotenv (override=True)

True

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.


Costs are here: https://platform.openai.com/docs/pricing#web-search

In [41]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must be 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself"

search_agent = Agent(
    name = "search agent",
    instructions=INSTRUCTIONS,
    model='gpt-4o-mini',
    tools = [WebSearchTool(search_context_size="low")],
    model_settings=ModelSettings(tool_choice="required")

)

In [42]:
OPEN_AI_KEY = os.getenv("OPENAI_API_KEY")

In [43]:
if OPEN_AI_KEY:
    print(f"Open AI key is found and starts with {OPEN_AI_KEY[:8]}")
else:
    print(f"Open AI key is not found. Check env and docs")


Open AI key is found and starts with sk-proj-


In [47]:
message = "Latest AI Agent framweorks in 2025"

with trace("Latest Agentic AI framweork"):
    result = await Runner.run(search_agent, message)
    
    
display(Markdown(result.final_output))



In 2025, several AI agent frameworks have emerged, each offering unique capabilities:

- **Agent Lightning**: A flexible framework enabling reinforcement learning-based training of large language models (LLMs) for any AI agent. It decouples agent execution from training, allowing seamless integration with existing agents and handling complex interactions like multi-agent scenarios and dynamic workflows. ([arxiv.org](https://arxiv.org/abs/2508.03680?utm_source=openai))

- **Cognitive Kernel-Pro**: An open-source, multi-module agent framework designed to democratize the development and evaluation of advanced AI agents. It focuses on curating high-quality training data across domains like web, file, code, and general reasoning, and introduces novel strategies for agent test-time reflection and voting to enhance robustness and performance. ([arxiv.org](https://arxiv.org/abs/2508.00414?utm_source=openai))

- **LightAgent**: A lightweight yet powerful agentic framework that integrates core functionalities such as memory, tools, and tree of thought, while maintaining an extremely lightweight structure. As a fully open-source solution, it seamlessly integrates with mainstream chat platforms, enabling developers to easily build self-learning agents. ([arxiv.org](https://arxiv.org/abs/2509.09292?utm_source=openai))

- **Kruti**: A multilingual AI agent and chatbot developed by the Indian company Ola Krutrim. It is designed to perform real-world tasks for users, such as booking taxis and ordering food, by integrating directly with various online services. It is notable for its ability to understand and respond in multiple Indian languages. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Kruti?utm_source=openai))

- **Model Context Protocol (MCP)**: An open standard introduced by Anthropic to standardize the way AI systems like LLMs integrate and share data with external tools, systems, and data sources. MCP provides a universal interface for reading files, executing functions, and handling contextual prompts, aiming to become a universal open standard for AI system connectivity and interoperability. ([en.wikipedia.org](https://en.wikipedia.org/wiki/Model_Context_Protocol?utm_source=openai))

These frameworks reflect the rapid advancements in AI agent development, offering diverse tools and protocols to enhance the capabilities and integration of AI systems. 

#### PLANNER AGENTS

Planner Agents - It will receive a query and come up with a handful of searches that it should run. Limiting the search to three for this purpose/prject due to costing

In [ ]:
Number_of_Searches = 3
#This is the number of searches it should run

INSTRUCTIONS = f"You are a helpful research assistant. Given a query , come up with a set of web services \
    to perform to best answer the query. Output {Number_of_Searches} terms to query for."

class WebSearchItem(BaseModel):
    reason:str
    "Your reasoning for why this search is important to the query"


    query:str
    "The search term to use for the web search"


class WebSearchPlan(BaseModel):

    searches: list(WebSearchItem)
    """A list of web searches to perform to best answer the query."""Agent




planner_agent = Agent(
    name = "PlannerAgent",
    model="gpt-4o-mini",
)